In [1]:
from bs4 import BeautifulSoup as BS
import urllib.request as req
import urllib.parse as pa
import pandas as pd
import math

In [2]:
# 평점 url 얻기 위한 url_open

def url_point_Open(movie_code):
    url = 'https://movie.naver.com/movie/bi/mi/point.nhn?code='+str(movie_code)
    url_open = req.urlopen(url)
    
    return url_open

In [3]:
def url_Open(sub_url, page_num):
    url = 'https://movie.naver.com' + sub_url +'&page='+ str(page_num)
    url_open = req.urlopen(url)
    
    return url_open

In [4]:
movie_code = 175366
soup = BS(url_point_Open(movie_code), 'html.parser')
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=175366&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [5]:
soup = BS(url_Open(ifr_url, 11), 'html.parser')

In [6]:
count = soup.find('strong', 'total').find('em')
count

<em>4,741</em>

In [7]:
count = int(count.string.replace(',', ''))

In [8]:
count

4741

In [9]:
loop_count = math.ceil(count/10)
loop_count

475

In [10]:
point = []
review = []
date = []

In [11]:
page_num = 1

while page_num  < loop_count+1:
    soup = BS(url_Open(ifr_url, page_num), 'html.parser')
    result = soup.find(class_='score_result')
    lis = result.find_all('li')
    print(page_num, end=' ')
    
    for lst in lis:
        count_span = len(lst.find(class_='score_reple').find_all('span'))
        count_icon = len(lst.find_all(class_='ico_viewer'))
        count_spo = len(lst.find_all(class_='text_spo'))
        point.append(int(lst.find(class_='star_score').find('em').get_text()))
#         print(count_span, count_icon, count_spo)
        
        if count_icon ==1 and count_span == 2:
            r = lst.find(class_='score_reple').find('p').get_text().strip().replace('\n', '').replace('\r','').replace('\t', '').replace('관람객', '')
        elif count_span == 2:
            r = lst.find(class_='score_reple').find('p').get_text().strip()
        elif count_icon ==1 and count_spo == 1:
            r = lst.find_all('span')[4].get_text().strip()
        elif count_icon == 1:
            r = lst.find(class_='score_reple').find('p').get_text().strip().replace('\n', '').replace('\r', '').replace('\t', '').replace('관람객', '')
        elif count_spo == 1:
            r = lst.find_all('span')[3].get_text().strip()
        else:
            r = lst.find_all('a')[0]['data-src']
        review.append(r)
        date.append(lst.find(class_='score_reple').find_all('em')[1].get_text())
            
    page_num += 1
    
    

    

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [12]:
df = pd.DataFrame(dict(zip(['날짜','리뷰', '평점'], [date, review, point])))
df

,날짜,리뷰,평점
0,2019.05.01 13:01,500만 찍으면 런닝맨 나온다고 했죠 500만 찍읍시다,10
1,2019.05.01 09:06,이광수 연기 이렇게 잘했었나..? 지루할틈 없이 두시간동안 웃다가 울다가. 볼만했다.,10
2,2019.05.01 09:07,신하균연기야 잘하는 건 알고 있었고 이광수의 재발견,10
3,2019.05.01 13:20,형아. 뜨거워. 후후후훟 ㅎ,10
4,2019.05.01 13:28,신하균 보러갔다 이광수에게 놀라고 온 영화,10
...,...,...,...
4736,2019.05.06 00:57,평점 보고 봤는데 영화 보다가 나왔다. 어벤저스에서 아이언맨 죽을때 눈물날만큼 평소...,1
4737,2019.05.02 20:42,이 영화를 가장 좋아하는 그룹은 20대 여자입니다,1
4738,2020.01.24 21:25,광수 팬들의 평점 올려주기 .. 연기가 너무 발연기라 그냥 광수고 리얼리티가 없슴,1
4739,2020.01.24 22:18,지적장애인 연기가 얼마나 어려운지 알겠다... 이광수 연기가 너무 어색해서 몰입이 ...,4


In [13]:
df.to_csv('movie.csv', encoding='utf-8')

In [14]:
# 이게 이렇게 작동하면 안되는데 작동이 된다... 허허